In [ ]:
from tkinter import *
from tkinter.filedialog import asksaveasfile, askopenfile
from tkinter.messagebox import showerror, showinfo
from PIL import ImageTk, Image
import rsa
import sqlite3

def normalize_string(strr: str):
    normalized_str = strr.strip().replace('\n', '')
    if normalized_str.isalpha():
        return normalized_str
    else:
        raise ValueError("String can only be english alphabet.")

def get_connection():
    return sqlite3.connect('database.db')

def get_users():
    result = None
    try:
        db = get_connection()
        cursor = db.cursor()
        query = "SELECT username FROM user_keys"
        cursor.execute(query)
        query_result = cursor.fetchall()
        result = [j for i in query_result for j in i]
    except sqlite3.DatabaseError as err:
        showerror(title="خطا", message="خطا در دریافت اطلاعات: "+str(err))
    except Exception as err:
        showerror(title="خطا", message="خطای ناشناخته ای رخ داد! لطفا با مدیر تماس بگیرید"+"\n"+str(err))
    finally:
        db.close()
        return result

def create_db_table(conn):
    try:
        cursor = conn.cursor()
        query = "CREATE TABLE IF NOT EXISTS user_keys (username VARCHAR(255) UNIQUE, public_key VARCHAR(255), private_key VARCHAR(255));"
        cursor.execute(query)
        conn.commit()
    except sqlite3.DatabaseError as err:
        showerror(title="خطا", message="خطا در ذخیره اطلاعات: "+str(err))
    except Exception as err:
        showerror(title="خطا", message="خطای ناشناخته ای رخ داد! لطفا با مدیر تماس بگیرید"+"\n"+str(err))

root = Tk()
root.title('رمزنگار - رمزگشا')
root.geometry('700x300')
root.resizable(width=False, height=False)
root.configure(bg='white')
font = ('B Yekan', 16)

buttonFrame = Frame(root, bg='white')
buttonFrame.pack(side=LEFT)

frame = Frame(root, bg='white')
frame.pack()

def save_key_file(name, content):
        f = asksaveasfile(mode='w', initialfile = f'{name}.pem', defaultextension=".pem", filetypes=[("Key Files","*.pem")])
        if f is None:
            return
        f.write(str(content))
        f.close()

def generateKeys(username: str):
    try:
        public_key, private_key = rsa.newkeys(1024)
        
        db = get_connection()
        cursor = db.cursor()
        
        create_db_table(db)
        
        query = "INSERT INTO user_keys VALUES (?, ?, ?);"
        cursor.execute(query, [normalize_string(username), public_key.save_pkcs1(), private_key.save_pkcs1()])

        db.commit()

        save_key_file('public', public_key.save_pkcs1())
        save_key_file('private', private_key.save_pkcs1())
        showinfo(title='عملیات موفقیت آمیز', message='عملیات با موفقیت انجام شد')
    except sqlite3.DatabaseError as err:
        showerror(title="خطا", message="خطا در ذخیره اطلاعات: "+str(err))
    except ValueError as err:
        showerror(title="خطا", message="خطا در ذخیره اطلاعات: "+str(err))
    except Exception as err:
        showerror(title="خطا", message="خطای ناشناخته ای رخ داد! لطفا با مدیر تماس بگیرید"+"\n"+str(err))
    finally:
        db.close()
        set_screen('home')
        
def encrypt(text, reciver_username):
    result = None
    try:
        db = get_connection()
        cursor = db.cursor()
        
        create_db_table(db)
        
        query = "SELECT * FROM user_keys WHERE username = ?"
        cursor.execute(query, [reciver_username])
        result = cursor.fetchone()
        
        if result:
            f = asksaveasfile(mode='w', defaultextension=".txt", filetypes=[("Text Documents","*.txt")])
            if f is None:
                return
            encrypted_txt = rsa.encrypt(text.encode('utf8'), rsa.PublicKey.load_pkcs1(result[1]))
            f.write(str(encrypted_txt))
            f.close()
            showinfo(title='عملیات موفقیت آمیز', message='عملیات با موفقیت انجام شد')
        else:
            showerror(title="خطا", message="خطا در دریافت اطلاعات"+"\n"+"کاربری با این نام وجود ندارد")
    
    except sqlite3.DatabaseError as err:
        showerror(title="خطا", message="خطا در دریافت اطلاعات: "+str(err))
        db.close()
    except Exception as err:
        showerror(title="خطا", message="خطای ناشناخته ای رخ داد! لطفا با مدیر تماس بگیرید"+"\n"+str(err))
    finally:
        db.close()
        
def decrypt(text, private_key):
    print(private_key)
    try:
        decryptedText = rsa.decrypt(text.encode('utf8'), rsa.PrivateKey.load_pkcs1(private_key))
        
        return decryptedText.decode('utf8')
    except Exception as err:
        showerror(title="خطا", message="خطای ناشناخته ای رخ داد! لطفا با مدیر تماس بگیرید"+"\n"+str(err))
        return ''

def display_screen(screen):
    global frame
    frame.destroy()
    frame = Frame(root, bg='white')
    frame.pack()
    print(screen)
    
    if screen == 'encrypt':     
        usersMenuChoice = StringVar(value="انتخاب کنید")
        userschoices = get_users()
        
        usersMenu = OptionMenu(frame, usersMenuChoice, "انتخاب کنید", *(userschoices))
        text2enc = Text(master=frame, width=50, height=5)
        btnEnc = Button(master=frame, font=font, text='رمزنگاری', command=lambda: encrypt(text2enc.get("1.0", END), usersMenuChoice.get())) # set a choose elements for public key users
        text2enc.pack()
        usersMenu.pack()
        btnEnc.pack()
    elif screen == 'decrypt':
        f = askopenfile(mode='r', defaultextension=".txt", filetypes=[("Text Documents","*.txt")])
        
        if f is None:
            showerror(title="خطا", message="لطفا فایلی را انتخاب کنید")
            set_screen('home')
        text2dec = f.read()
        
        f.close()
        
        f = askopenfile(mode='rb', defaultextension=".pem", filetypes=[("Key Files","*.pem")])
        
        if f is None:
            showerror(title="خطا", message="لطفا فایلی را انتخاب کنید")
            set_screen('home')
        private_key = f.read()
        
        f.close()
        
        decryptedOutputTxt = StringVar(value=decrypt(text2dec, private_key))
        decryptedOutput = Label(frame, width=50, height=5, background='white', textvariable=decryptedOutputTxt, font=('comic sans ms', 16), anchor='nw')
        
        decryptedOutput.pack()
        
    elif screen == 'generate':
        userInputTxt = StringVar(value='')
        userInput = Entry(master=frame, width=50, font=('comic sans ms', 16), textvariable=userInputTxt)
        btnGenerate = Button(master=frame, font=font, text='ساخت کلید', command=lambda: generateKeys(userInputTxt.get()))
        userInput.pack()
        btnGenerate.pack()
    elif screen == "home":
        txt = "F2 - رمزنگاری"+"\n"+"F3 - رمزگشایی"+"\n"+"F4 - ساخت کلید"+"\n"+"Esc - خروج"
        label = Label(frame, text=txt, font=font, bg='white')
        label.pack(side=LEFT, padx=30, pady=30)
    else:
        showerror(title="خطا", message="خطای ناشناخته ای رخ داد! لطفا با مدیر تماس بگیرید")
    
    root.mainloop()

def set_screen(screen):
    display_screen(screen)

def closeForm(x=''):
    root.destroy()

btnEncryptImage = Image.open('img/encryptbtn.png').resize((200, 100))
btnDecryptImage = Image.open('img/decryptbtn.png').resize((200, 100))
btnGenerateImage = Image.open('img/genkeysbtn.png').resize((200, 100))
btnExitImage = Image.open('img/exitbtn.png').resize((200, 100))

btnEncryptImg = ImageTk.PhotoImage(btnEncryptImage)
btnDecryptImg = ImageTk.PhotoImage(btnDecryptImage)
btnGenerateImg = ImageTk.PhotoImage(btnGenerateImage)
btnExitImg = ImageTk.PhotoImage(btnExitImage)

btnEncrypt = Button(buttonFrame, text='رمزنگاری', font=font, image=btnEncryptImg, border='0', bg='white', height=60, command=lambda: set_screen('encrypt'))
btnDecrypt = Button(buttonFrame, text='رمزگشایی', font=font, image=btnDecryptImg, border='0', bg='white', height=60, command=lambda: set_screen('decrypt'))
btnGenerate = Button(buttonFrame, text='ساخت کلید', font=font, image=btnGenerateImg, border='0', bg='white', height=60, command= lambda: set_screen('generate'))
btnExit = Button(buttonFrame, text='خروج از برنامه', font=font, image=btnExitImg, border='0', bg='white', height=60, command=closeForm, )

btnExit.pack(side=BOTTOM, anchor='w', padx=5, pady=5)
btnEncrypt.pack(side=BOTTOM, anchor='w', padx=5)
btnDecrypt.pack(side=BOTTOM, anchor='w', padx=5, pady=5)
btnGenerate.pack(side=BOTTOM, anchor='w', padx=5)

root.bind('<F2>', lambda x='': set_screen('encrypt'))
root.bind('<F3>', lambda x='': set_screen('decrypt'))
root.bind('<F4>', lambda x='': set_screen('generate'))
root.bind('<Escape>', closeForm)

set_screen('home')

## TODO:
1. [x] Find the command for remove frame elements
2. [x] lock the window size
3. [x] better buttons
4. [x] set hotkey events
5. [x] Generate Keys
6. [x] Encrypt Message
7. [ ] Decrypt Message
8. [x] Exit
9. [ ] Remove 'print' logs
10. [ ] Modulize Project